<a href="https://colab.research.google.com/github/cmannnn/california_housing/blob/main/california_housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning Housing Corp. - California Housing

The first task you are asked to perform is to build a model of housing prices in California using the California cen‐ sus data. This data has metrics such as the population, median income, median hous‐ ing price, and so on for each block group in California. Block groups are the smallest geographical unit for which the US Census Bureau publishes sample data (a block group typically has a population of 600 to 3,000 people). We will just call them “dis‐ tricts” for short.

**Your task is to predict median house values in Californian districts, given a number of features from these districts.**

#Imports

In [11]:
# imports
import sys
print(f'Using sys version: {sys.version_info}')

import requests

import csv

import os

import matplotlib.pyplot as plt
import matplotlib
plt.style.use('fivethirtyeight')
%matplotlib inline
print(f'Using matplotlib version: {matplotlib.__version__}')

import numpy as np
print(f'Using numpy version: {np.__version__}')

import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
print(f'Using pandas version: {pd.__version__}')

import seaborn as sns
print(f'Using seaborn version: {sns.__version__}')

import warnings
warnings.simplefilter('ignore')

Using sys version: sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)
Using matplotlib version: 3.2.2
Using numpy version: 1.19.4
Using pandas version: 1.1.5
Using seaborn version: 0.11.0


In [25]:
# mounting google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [26]:
# changing drive location
os.chdir('/content/gdrive/MyDrive/Colabby Krabby Paddy')

In [ ]:
# use URLLIB to download from link 
urllib.request.urlopen('https://github.com/ageron/handson-ml2/blob/master/datasets/housing/housing.csv')